## setup and load dataset 

In [ ]:
import pandas as pd

# Load QM9 dataset
url = 'https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv'
df = pd.read_csv(url)

# Inspect basic info
print("Columns:", df.columns.tolist())
#smiles (Molecule string), gap (HOMO-LUMO gap), mu (dipole moment)
df = df[['smiles', 'gap', 'mu']]  # Keep only what's needed
df = df.dropna()

print(df.head())


Columns: ['mol_id', 'smiles', 'A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom', 'g298_atom']
  smiles     gap      mu
0      C  0.5048  0.0000
1      N  0.3399  1.6256
2      O  0.3615  1.8511
3    C#C  0.3351  0.0000
4    C#N  0.3796  2.8937


## convert SMILES to SELFIES

In [ ]:
# Safely encode SMILES to SELFIES, allowing invalids to return None
import selfies as sf #Self-Referencing Embedded Strings (Used to tokenize)
import rdkit as rd #open-source toolkit for cheminformatics
def safe_encode(smiles):
    try:
        return sf.encoder(smiles, strict=False)
    except sf.EncoderError:
        return None

df['selfies'] = df['smiles'].apply(safe_encode)

# Drop rows where encoding failed
df = df[df['selfies'].notnull()]


## tokenize SELFIES

In [ ]:
#Tokenize to split the molecules into atoms
# Build vocabulary to map the tokens to
all_tokens = set() #stores unique tokens from all SELFIES
tokenized = [] #List of token sequences
max_len = 0

for s in df['selfies']:
    tokens = list(sf.split_selfies(s)) #Splits each SELFIES string into a list of tokens.
    all_tokens.update(tokens) #Updates vocabulary
    tokenized.append(tokens)
    max_len = max(max_len, len(tokens)) #Keeps track of the maximum sequence length.


# Index maps
token2idx = {tok: i + 2 for i, tok in enumerate(sorted(all_tokens))}
token2idx["<PAD>"] = 0 #Padding
token2idx["<SOS>"] = 1 #Start of sequence
idx2token = {i: t for t, i in token2idx.items()} #maps index back to its token
vocab_size = len(token2idx) #Total number of unique tokens

# Encode sequences
def encode(tokens):
    ids = [token2idx["<SOS>"]] + [token2idx[t] for t in tokens]
    return ids + [0] * (max_len - len(tokens))

encoded = [encode(t) for t in tokenized] #Encodes all tokenized sequences



## PREPARE DATA FOR TRAINING 

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class MoleculeDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = torch.tensor(sequences, dtype=torch.long)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.sequences[idx]

dataset = MoleculeDataset(encoded)
loader = DataLoader(dataset, batch_size=64, shuffle=True)


## build VAE model 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
    #Token embedding size --> used as input for the encoder
    def __init__(self, vocab_size, emb_dim=128, hidden_dim=256, latent_dim=64):
        super(VAE, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim) #Turns token indices into vectors

        # Encoder
        self.encoder_rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True) #Compresses input to hidden vector
        self.fc_mu = nn.Linear(hidden_dim, latent_dim) #Mean of latent distribution
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim) #log-variance (Uncertainty of distribution)

        # Decoder
        self.latent2hidden = nn.Linear(latent_dim, hidden_dim)
        self.decoder_rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True) #Reads in <SOS> tokens
        self.output = nn.Linear(hidden_dim, vocab_size) #GRU output --> Predicted tokens

    def encode(self, x):
        x = self.embedding(x)
        _, h = self.encoder_rnn(x)
        h = h.squeeze(0)
        return self.fc_mu(h), self.fc_logvar(h)

    #Chooses a random point z using mu and logvar, where z is a compressed version of the input (Used to generate new SELFIES)
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    #Decodes z to a sequence of logits
    def decode(self, z, seq_len):
        hidden = self.latent2hidden(z).unsqueeze(0)
        inputs = torch.full((z.size(0), seq_len), token2idx['<SOS>'], dtype=torch.long).to(z.device)
        emb = self.embedding(inputs)
        out, _ = self.decoder_rnn(emb, hidden)
        return self.output(out)

    def forward(self, x): #VAE forward pass
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        logits = self.decode(z, x.size(1)) #raw, unprocessed output of a model before it's turned into a probability
        return logits, mu, logvar


## train the VAE 

In [ ]:
import torch.optim as optim
from tqdm import tqdm  # for progress bar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(vocab_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def loss_fn(recon_logits, target, mu, logvar):
    #Compares output vs target
    recon_loss = F.cross_entropy(recon_logits.view(-1, vocab_size), target.view(-1), ignore_index=0)
    #Ensures latent space is Gaussian (Normal distribution)
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / target.size(0)
    return recon_loss + kl_loss, recon_loss, kl_loss

# Print dataset and batch info
print(f"Dataset size: {len(dataset)}")
print(f"Number of batches: {len(loader)}")

# Training loop
for epoch in range(1, 11):  # 1 to 10 inclusive
    model.train()
    total_loss = 0
    total_recon = 0
    total_kl = 0

    print(f"\nEpoch {epoch}")

    #Forward → loss → backprop → optimizer step and Tracks total loss.
    for batch_idx, (x, y) in enumerate(tqdm(loader)):
        x, y = x.to(device), y.to(device)
        logits, mu, logvar = model(x)
        loss, recon_loss, kl_loss = loss_fn(logits, y, mu, logvar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_recon += recon_loss.item()
        total_kl += kl_loss.item()

    print(f"Epoch {epoch}: Total Loss = {total_loss/len(loader):.4f} | Recon = {total_recon/len(loader):.4f} | KL = {total_kl/len(loader):.4f}")


Dataset size: 133885
Number of batches: 2092

Epoch 1


100%|██████████| 2092/2092 [04:21<00:00,  8.01it/s]


Epoch 1: Total Loss = 1.6980 | Recon = 1.6954 | KL = 0.0025

Epoch 2


100%|██████████| 2092/2092 [04:56<00:00,  7.04it/s]


Epoch 2: Total Loss = 1.6747 | Recon = 1.6745 | KL = 0.0002

Epoch 3


...

Epoch 8: Total Loss = 1.6613 | Recon = 1.6613 | KL = 0.0000

Epoch 9


100%|██████████| 2092/2092 [10:09<00:00,  3.43it/s]


Epoch 9: Total Loss = 1.6614 | Recon = 1.6610 | KL = 0.0004

Epoch 10


100%|██████████| 2092/2092 [04:17<00:00,  8.13it/s]

Epoch 10: Total Loss = 1.6608 | Recon = 1.6608 | KL = 0.0000


## GENERATE NEW MOLECULES 

In [ ]:
import torch.nn.functional as F
import selfies as sf

# This should have been calculated during tokenization
max_len = max(len(t) for t in tokenized)  # tokenized = list of token lists
MAX_LEN = max_len  # define global name if needed

#Samples 100 random points in latent space and decodes each into a sequence of tokens.
#Converts token sequences → SELFIES → SMILES.
def generate_molecules(model, num_samples=100, latent_dim=64, max_len=MAX_LEN, device='cpu'):
    model.eval()
    z = torch.randn(num_samples, latent_dim).to(device)
    
    with torch.no_grad():
        logits = model.decode(z, max_len)  # (batch, seq_len, vocab_size)
        probs = F.softmax(logits, dim=-1)  # Turn logits into probabilities
        
        # Multinomial sampling from the probability distribution
        sampled_ids = torch.zeros_like(logits.argmax(dim=-1), dtype=torch.long)
        for i in range(logits.size(1)):  # for each position in sequence
            sampled_ids[:, i] = torch.multinomial(probs[:, i], num_samples=1).squeeze()

    # Convert token IDs → SELFIES → SMILES
    generated_selfies = []
    for seq in sampled_ids.cpu().numpy():
        tokens = [idx2token.get(i, '') for i in seq if i > 1]
        selfies_str = "".join(tokens)
        generated_selfies.append(selfies_str)

    generated_smiles = [sf.decoder(s) for s in generated_selfies]
    return generated_smiles


## VALIDATE & EVALUATE MOLECULES 

In [ ]:
# STEP 1: Generate 100 molecules using the improved sampling method
generated_smiles = generate_molecules(
    model,
    num_samples=100,
    latent_dim=64,
    max_len=max_len,
    device=device
)

# STEP 2: Run your evaluation block (validity, uniqueness, novelty, qed, logP)
from rdkit import Chem
from rdkit.Chem import QED, Crippen

valid_smiles = []
seen_set = set()
novel_set = set()
qed_list = [] #Drug likeness
logp_list = [] #Hydrophobicity

train_smiles_set = set(df['smiles']) #Used to determine if it's new

print("=== Generated Molecule Evaluation ===")

#Checks which SMILES are chemically valid
for i, smi in enumerate(generated_smiles):
    mol = Chem.MolFromSmiles(smi)
    if mol:
        canonical = Chem.MolToSmiles(mol)
        valid_smiles.append(canonical)

        qed = QED.qed(mol)
        logp = Crippen.MolLogP(mol)
        qed_list.append(qed)
        logp_list.append(logp)

        print(f"{i+1}: ✅ Valid SMILES - {canonical}")
        print(f"    QED: {qed:.3f}, logP: {logp:.2f}")

        if canonical not in train_smiles_set:
            novel_set.add(canonical)

        seen_set.add(canonical)
    else:
        print(f"{i+1}: ❌ Invalid SMILES")

# STEP 3: Evaluation metrics
total = len(generated_smiles)
valid = len(valid_smiles)
unique = len(seen_set)
novel = len(novel_set)

validity = 100 * valid / total
uniqueness = 100 * unique / valid if valid else 0
novelty = 100 * novel / valid if valid else 0

print("\n=== Summary ===")
print(f"Total Generated: {total}")
print(f"✅ Valid: {valid} ({validity:.2f}%)")
print(f"🔁 Unique: {unique} ({uniqueness:.2f}%)")
print(f"🆕 Novel: {novel} ({novelty:.2f}%)")


=== Generated Molecule Evaluation ===
1: ✅ Valid SMILES - C1=C2CC=C3C1CC3ON2
    QED: 0.502, logP: 1.12
2: ✅ Valid SMILES - C1=COON1
    QED: 0.407, logP: -0.08
3: ✅ Valid SMILES - C1=NCCN=CCC1
    QED: 0.443, logP: 0.92
4: ✅ Valid SMILES - C=CCCC=COC#N
    QED: 0.248, logP: 1.96
5: ✅ Valid SMILES - CC1NCCCOOC1C
    QED: 0.509, logP: 0.70
6: ✅ Valid SMILES - CCN=CCN(N)OO
    QED: 0.238, logP: -0.34
7: ✅ Valid SMILES - C1=C2COCCC12
    QED: 0.408, logP: 0.96
8: ✅ Valid SMILES - C1=C2C3CC(C1)C2C3
    QED: 0.414, logP: 1.97
9: ✅ Valid SMILES - CCC=C=N
    QED: 0.447, logP: 1.20
10: ✅ Valid SMILES - CC=CCCCC=COCF
    QED: 0.327, logP: 3.19
11: ✅ Valid SMILES - C1=C2CC12
    QED: 0.360, logP: 0.95
12: ✅ Valid SMILES - C1=CC2OCC2CCCCC1
    QED: 0.485, logP: 2.52
13: ✅ Valid SMILES - N1=NC2OC12
    QED: 0.373, logP: 0.13
14: ✅ Valid SMILES - CCC=C=NCNC1=NC1
    QED: 0.567, logP: 0.58
15: ✅ Valid SMILES - C1=C2C3=C1N2OCC3
    QED: 0.453, logP: 0.79
16: ✅ Valid SMILES - CCCCOCOCCC#N
    QED: 0.